In [52]:
import pandas as pd
import os
import seaborn as sns
from skbio.diversity import alpha_diversity
from skbio.diversity import beta_diversity
from skbio import TreeNode
from io import StringIO
from skbio.stats.ordination import pcoa

import cx_Oracle

SERVER = 'dboracle.itk.ppke.hu'
PORT = 1521
SERVICE = 'rsc.itk.ppke.hu'
USER = 'metagenome'
PASSWORD = '.'

class Connection():

    def __init__(self):
        dsn_tns = cx_Oracle.makedsn(SERVER, PORT, service_name=SERVICE)
        self.connection = cx_Oracle.connect(USER, PASSWORD, dsn_tns)

    def get_connection(self):
        return self.connection

    def select(self, sql):
        return pd.read_sql(sql, con=self.connection)

con = Connection()

In [53]:
path = '//gfs/data/curated_metagenomes_kraken2/'

files = {}
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        for r1, d1, f1 in os.walk(os.path.join(path,directory)):
            for file in f1:
                if '.report.out' in file:
                    if directory in files:
                        files[directory].append(os.path.join(r1, file))
                    else:
                        files[directory] = [(os.path.join(r1, file))]

In [54]:
def get_data_for_sample(path_to_file):
    dataset = path_to_file.split('/')[-2]
    df = pd.read_csv(path_to_file, delimiter = '\t', 
                     names=["Abundance", "Root_count", "Taxon_count", "Rank", "NCBI_ID", "Name"])
    sample_id = path_to_file.split('/')[-1].split('.')[0]
    df['Sample_ID'] = sample_id
    df['Dataset'] = dataset
    not_found_files = set()
    return df

In [63]:
def join_metadata(df, dataset_name):
    SQL_METADATA = """
        select
            DATASET_NAME,
            STUDY_CONDITION,
            BODY_SITE,
            SUBJECTID,
            NCBI_ACCESSION,
            COUNTRY,
            DISEASE,
            GENDER,
            AGE,
            AGE_CATEGORY,
            NUMBER_READS
        from METADATA
        where dataset_name = '{dataset_name}'
        """.replace("{dataset_name}", dataset_name)
    df_metadata = con.select(SQL_METADATA)
    
    df["DATASET_NAME"] = df["Dataset"]
    merged = df.merge(df_metadata, on='DATASET_NAME')
    
    col_to_idx = {}
    for i in range(len(merged.columns)):
        col_to_idx[merged.columns[i]] = i
    
    joined = merged.loc[[row[1][col_to_idx["Sample_ID"]] in row[1][col_to_idx["NCBI_ACCESSION"]] 
                        for row in merged.iterrows()]]
    
    output = joined[["Sample_ID", "Dataset", "Rank", "Name", "Abundance", "", "Taxon_count", "STUDY_CONDITION", 
                     "BODY_SITE", "SUBJECTID", "COUNTRY", "DISEASE", "AGE", "AGE_CATEGORY", "NUMBER_READS"]]
    output = output.reset_index()
    output = output.drop(columns="index")
    return output

In [56]:
dataset_dfs = {}
result = joined
for dataset_name, filelist in files.items():
    if dataset_name = "AsnicarF_2017":
        df_list = []
        for file in filelist:
            df = get_data_for_sample(file)
            if df is not None:
                df_list.append(get_data_for_sample(file))
        df_dataset = pd.concat(df_list)
        joined = join_metadata(df_dataset, dataset_name)

RampelliS_2015  kész.
AsnicarF_2017  kész.


KeyboardInterrupt: 

1. Percentage of fragments covered by the clade rooted at this taxon
2. Number of fragments covered by the clade rooted at this taxon
3. Number of fragments assigned directly to this taxon
4. A rank code, indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom,
   (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, or (S)pecies.
   Taxa that are not at any of these 10 ranks have a rank code that is
   formed by using the rank code of the closest ancestor rank with
   a number indicating the distance from that rank.  E.g., "G2" is a
   rank code indicating a taxon is between genus and species and the
   grandparent taxon is at the genus rank.
5. NCBI taxonomic ID number
6. Indented scientific name

In [51]:
df_union_all.loc[df_union_all["Rank"] == 'R1']

NameError: name 'df_union_all' is not defined

In [50]:
3635181/77841428*100

4.669982416047146

In [55]:
dataset_name = "RampelliS_2015"

In [45]:
output = join_metadata(df_union_all, dataset_name)

In [56]:
path_to_output = os.path.join(path, dataset_name + ".csv")

In [57]:
path_to_output

'//gfs/data/curated_metagenomes_kraken2/RampelliS_2015.csv'

In [59]:
with open(path_to_output, "w") as f: 
    f.write(output.to_csv())

In [46]:
output.head()

,Sample_ID,Dataset,Rank,Name,Abundance,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,AGE,AGE_CATEGORY
0,SRR1930123,RampelliS_2015,U,unclassified,89.48,control,stool,H17,TZA,none,23,adult
1,SRR1930123,RampelliS_2015,R,root,10.52,control,stool,H17,TZA,none,23,adult
2,SRR1930123,RampelliS_2015,R1,cellular organisms,9.99,control,stool,H17,TZA,none,23,adult
3,SRR1930123,RampelliS_2015,D,Bacteria,9.80,control,stool,H17,TZA,none,23,adult
4,SRR1930123,RampelliS_2015,D1,Terrabacteria group,5.51,control,stool,H17,TZA,none,23,adult


In [59]:
SQL_METADATA = """
select
    DATASET_NAME,
    STUDY_CONDITION,
    BODY_SITE,
    SUBJECTID,
    NCBI_ACCESSION,
    COUNTRY,
    DISEASE,
    GENDER,
    AGE,
    AGE_CATEGORY
from METADATA
where dataset_name = 'RampelliS_2015'
"""
df_metadata = con.select(SQL_METADATA)

In [60]:
df_metadata.head()

,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
0,RampelliS_2015,control,stool,H1,SRR1927149,TZA,none,female,43,adult
1,RampelliS_2015,control,stool,H2,SRR1930133,TZA,none,male,37,adult
2,RampelliS_2015,control,stool,H3,SRR1930145,TZA,none,male,22,adult
3,RampelliS_2015,control,stool,H4,SRR1930149,TZA,none,male,24,adult
4,RampelliS_2015,control,stool,H5,SRR1930176,TZA,none,female,70,senior


In [61]:
df_union_all["DATASET_NAME"] = df_union_all["Dataset"]
merged = df_union_all.merge(df_metadata, on='DATASET_NAME')

NameError: name 'df_union_all' is not defined

In [22]:
col_to_idx = {}
for i in range(len(merged.columns)):
    col_to_idx[merged.columns[i]] = i

In [23]:
joined = merged.loc[[row[1][col_to_idx["Sample_ID"]] in row[1][col_to_idx["NCBI_ACCESSION"]] for row in merged.iterrows()]]

In [24]:
joined.head()

,Abundance,Root_count,Taxon_count,Rank,NCBI_ID,Name,Sample_ID,Dataset,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
16,89.48,33178082,33178082,U,0,unclassified,SRR1930123,RampelliS_2015,RampelliS_2015,control,stool,H17,SRR1930123,TZA,none,male,23,adult
54,10.52,3901358,165809,R,1,root,SRR1930123,RampelliS_2015,RampelliS_2015,control,stool,H17,SRR1930123,TZA,none,male,23,adult
92,9.99,3704939,3522,R1,131567,cellular organisms,SRR1930123,RampelliS_2015,RampelliS_2015,control,stool,H17,SRR1930123,TZA,none,male,23,adult
130,9.80,3635181,178284,D,2,Bacteria,SRR1930123,RampelliS_2015,RampelliS_2015,control,stool,H17,SRR1930123,TZA,none,male,23,adult
168,5.51,2041509,33524,D1,1783272,Terrabacteria group,SRR1930123,RampelliS_2015,RampelliS_2015,control,stool,H17,SRR1930123,TZA,none,male,23,adult


minta, projekt, taxonomy_level, baci, abundance

In [42]:
output = joined[["Sample_ID", "Dataset", "Rank", "Name", "Abundance", "STUDY_CONDITION", "BODY_SITE", "SUBJECTID",
                 "COUNTRY", "DISEASE", "AGE", "AGE_CATEGORY"]]
output = output.reset_index()
output = output.drop(columns="index")

In [43]:
output.head()

,Sample_ID,Dataset,Rank,Name,Abundance,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,AGE,AGE_CATEGORY
0,SRR1930123,RampelliS_2015,U,unclassified,89.48,control,stool,H17,TZA,none,23,adult
1,SRR1930123,RampelliS_2015,R,root,10.52,control,stool,H17,TZA,none,23,adult
2,SRR1930123,RampelliS_2015,R1,cellular organisms,9.99,control,stool,H17,TZA,none,23,adult
3,SRR1930123,RampelliS_2015,D,Bacteria,9.80,control,stool,H17,TZA,none,23,adult
4,SRR1930123,RampelliS_2015,D1,Terrabacteria group,5.51,control,stool,H17,TZA,none,23,adult


In [64]:
df_metadata_distinct = pd.DataFrame(df_metadata.NCBI_ACCESSION.str.split(';').tolist(), 
                                        index = df_metadata.index).stack()

In [65]:
df_metadata_distinct.head()

0  0    SRR1927149
1  0    SRR1930133
2  0    SRR1930145
3  0    SRR1930149
4  0    SRR1930176
dtype: object

In [66]:
df_metadata.head()

,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
0,RampelliS_2015,control,stool,H1,SRR1927149,TZA,none,female,43,adult
1,RampelliS_2015,control,stool,H2,SRR1930133,TZA,none,male,37,adult
2,RampelliS_2015,control,stool,H3,SRR1930145,TZA,none,male,22,adult
3,RampelliS_2015,control,stool,H4,SRR1930149,TZA,none,male,24,adult
4,RampelliS_2015,control,stool,H5,SRR1930176,TZA,none,female,70,senior


In [39]:
SQL_METADATA = """
select
    DATASET_NAME,
    STUDY_CONDITION,
    BODY_SITE,
    SUBJECTID,
    NCBI_ACCESSION,
    COUNTRY,
    DISEASE,
    GENDER,
    AGE,
    AGE_CATEGORY
from METADATA
where dataset_name = 'QinJ_2012'
"""
df_metadata = con.select(SQL_METADATA)

In [62]:
df_metadata.head()

,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
0,RampelliS_2015,control,stool,H1,SRR1927149,TZA,none,female,43,adult
1,RampelliS_2015,control,stool,H2,SRR1930133,TZA,none,male,37,adult
2,RampelliS_2015,control,stool,H3,SRR1930145,TZA,none,male,22,adult
3,RampelliS_2015,control,stool,H4,SRR1930149,TZA,none,male,24,adult
4,RampelliS_2015,control,stool,H5,SRR1930176,TZA,none,female,70,senior


In [47]:
df_metadata = df_metadata.loc[[x is not None for x in df_metadata.NCBI_ACCESSION]]
df_metadata_distinct = pd.DataFrame(df_metadata.NCBI_ACCESSION.str.split(';').tolist(),
                                    index = [df_metadata.DATASET_NAME,
                                             df_metadata.STUDY_CONDITION, df_metadata.BODY_SITE,
                                            df_metadata.SUBJECTID, df_metadata.COUNTRY,
                                            df_metadata.DISEASE, df_metadata.GENDER, 
                                            df_metadata.AGE, df_metadata.AGE_CATEGORY]).stack()

In [48]:
df_metadata_distinct.head()

DATASET_NAME  STUDY_CONDITION  BODY_SITE  SUBJECTID  COUNTRY  DISEASE  GENDER  AGE   AGE_CATEGORY   
QinJ_2012     T2D              stool      DLF001     CHN      T2D      female  59.0  adult         0    SRR341581
                                          DLF002     CHN      T2D      female  43.0  adult         0    SRR341582
                                          DLF003     CHN      T2D      female  46.0  adult         0    SRR341583
                                          DLF004     CHN      T2D      female  25.0  adult         0    SRR341584
                                          DLF005     CHN      T2D      female  60.0  adult         0    SRR341585
dtype: object

In [25]:
new_df = df_metadata_distinct.reset_index([0, 'DATASET_NAME', 'STUDY_CONDITION',
    'BODY_SITE',
    'SUBJECTID',
    'COUNTRY',
    'DISEASE',
    'GENDER',
    'AGE',
    'AGE_CATEGORY'])
new_df['NCBI_ACCESSION'] = new_df[0]

In [28]:
new_df = new_df.drop(columns=0)

In [29]:
new_df.head()

,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY,NCBI_ACCESSION
0,BritoIL_2016,control,oralcavity,M1.6,FJI,none,male,6,child,SRR2231781
1,BritoIL_2016,control,oralcavity,M1.6,FJI,none,male,6,child,SRR2243536
2,BritoIL_2016,control,oralcavity,M1.6,FJI,none,male,6,child,SRR2243582
3,BritoIL_2016,control,oralcavity,M1.6,FJI,none,male,6,child,SRR2243651
4,BritoIL_2016,control,oralcavity,M1.6,FJI,none,male,6,child,SRR2244452


In [ ]:
df["DATASET_NAME"] = df["Dataset"]
df["NCBI_ACCESSION"] = df["SampleId"]

merged = df.merge(df_metadata, on='NCBI_ACCESSION')

In [38]:
df_metadata_distinct = pd.DataFrame(df_metadata.NCBI_ACCESSION.str.split(';').tolist(),
                                    index = [df_metadata.DATASET_NAME,
                                             df_metadata.STUDY_CONDITION, df_metadata.BODY_SITE,
                                            df_metadata.SUBJECTID, df_metadata.COUNTRY,
                                            df_metadata.DISEASE, df_metadata.GENDER,
                                            df_metadata.AGE, df_metadata.AGE_CATEGORY]).stack()